# Tutorial inicial 

A continuación desarrollaremos un tutorial introductorio para sistemas de recomendación disponible en: https://www.datacamp.com/community/tutorials/recommender-systems-python


In [89]:
import pandas as pd
import numpy as np 

## Recomendaciones simples

In [90]:
#cargamos los datos que usaremos
metadata = pd.read_csv('C:/Users/gpabl/Documents/git/proyectomat282/datos/movies_metadata.csv', low_memory=False)

In [91]:
metadata.head(2) #visualizamos la 2 primeras lineas del cuadro.

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


Se ocupa una metrica que mezcla el numero de votantes con la calificacion obtenida para asi asegurar que una pelicula con muchos votantes tenga un valor mayor que una pelicula con muy pocos votantes y ambas con igual calificación. una metrica que cumple con estos requisitos es $$ WeightedRating = WR = WR(v,R,m.C) = \left(\dfrac{v}{v+m}R\right) + \left(\dfrac{m}{v+m}C\right) $$ donde $v$ es el numero de votos de la pelicula, $m$ es el numero minimo de votos necesarios para ser incluida en el cuadro (a elección), $R$ es la calificacion promedio de la pelicula y $C$ es ¿el promedio de votos de todo el informe?.

los valores de $v,R$ estan directamente en la tabla, $C$ lo podemos calcular y $m$ es un valor a elegir,correspondiente por ejemplo a un percentil dado. en el tutorial usan percentil=0.9. De esta manera 

In [92]:
C = metadata['vote_average'].mean() #calculamos el valor de C
percentil = 0.9 # definimos el percentil deseado
m = metadata['vote_count'].quantile(percentil) #Calculamos el percentil antes definido.
display(C)
display(m)



5.618207215134185

160.0

In [93]:
# cramos un dataframe auxiliar con las peliculas que cumplen con el minimo de votos
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]


El numero de peliculas en el dataframe filtrado es mucho menor que el original pues 

In [94]:
display(q_movies.shape)
display(metadata.shape)

(4555, 24)

(45466, 24)

Posteriormente hacemos una funcion que nos permita calcular la metrica antes vista

In [95]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

y posteriormente agregamos una columna llamada "score" al data frame auxiliar q_movies  utilizando la funcion anterior

In [96]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [97]:
q_movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,7.640253
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,6.820293


Y lo ordenamos de forma descendiente respecto al score.

In [98]:
q_movies = q_movies.sort_values('score', ascending=False)

In [99]:
#filtramos convenientemente y visualizamos las 15 mejores peliculas
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


## Content-Based Recommender

### Plot Description Based Recommender

se recomendaran peliculas de acuerdo a la similidaridad con una pelicula en particular. ocupamos la descripcion de la trama como factor de similaridad. Por eso observe algunas descripciones 

In [100]:
metadata['overview'].head(3)

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
Name: overview, dtype: object

Se introduce la calificacion TF-IDF  que es la frecuencia con que aparece una palabra en un documento, ponderada por el número de documentos en los que aparece. con esto se contruye una matriz donde cada columna representa una palabra en el vocabulario general (todas las palabras que aparecen en al menos un documento), y cada columna (fila?) representa una película. Para lo anterior hacemos uso de la libreria scikit-learn que ya tiene implementado esto 


In [101]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75827)

In [102]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[5005:5010] #no entendi 

['avalon', 'avant', 'avanthika', 'avanti', 'avaracious']

posteriormente dada la matriz anterior debemos definir una metrica. en este caso se utiliza cosine similarity() dada por $$\cos(X,Y) = \dfrac{XY^T}{||X||\cdot||Y||}$$

La cual dada el contexto podemos utilizar la funcion linear_kernel() ya implementada

In [103]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
dim = 1000 ## dimension de la matriz, la achicamos por temas de memoria
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix[0:dim,0:dim], tfidf_matrix[0:dim,0:dim]) 

In [104]:
#dimensiones de la matriz
cosine_sim.shape

(1000, 1000)

In [105]:
#cosine_sim[1] 
#Construimos una serie con los indices y titulos como indices
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [106]:
#visualizamos 
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

ahora se procede a construir las recomendaciones dado el titulo de una pelicula y para lo cual 
- se obtiene el indice de la pelicula
- Obtenga la lista de puntuaciones de similitud de coseno para esa película en particular con todas las películas. Conviértalo en una lista de tuplas donde el primer elemento es su posición y el segundo es la puntuación de similitud.
- Ordene la lista de tuplas antes mencionada en función de las puntuaciones de similitud; es decir, el segundo elemento.
- Obtenga los 10 elementos principales de esta lista. Ignore el primer elemento, ya que se refiere a sí mismo (la película más similar a una película en particular es la película en sí).
- Devuelve los títulos correspondientes a los índices de los elementos superiores.


In [107]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

ahora ya tenemos la funcion. probemosla con un ejemplo. 

In [108]:
get_recommendations('Waiting to Exhale',cosine_sim[0:dim,0:dim])

1                         Jumanji
2                Grumpier Old Men
3               Waiting to Exhale
4     Father of the Bride Part II
5                            Heat
6                         Sabrina
7                    Tom and Huck
8                    Sudden Death
9                       GoldenEye
10         The American President
Name: title, dtype: object

### Recomendación basada en créditos, géneros y palabras clave


La calidad de su recomendador aumentaría con el uso de mejores metadatos y capturando más de los detalles más finos. Eso es precisamente lo que vas a hacer en esta sección. Construirá un sistema de recomendación basado en los siguientes metadatos: los 3 actores principales, el director, los géneros relacionados y las palabras clave de la trama de la película.

Las palabras clave, el elenco y los datos del equipo no están disponibles en su conjunto de datos actual, por lo que el primer paso sería cargarlos y fusionarlos en su DataFrame principal metadata

In [109]:
# Load keywords and credits

credits = pd.read_csv('C:/Users/gpabl/Documents/git/proyectomat282/datos/credits.csv')
keywords = pd.read_csv('C:/Users/gpabl/Documents/git/proyectomat282/datos/keywords.csv')

# Remove rows with bad IDs.
metadata = metadata.drop([19730, 29503, 35587])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [110]:
#visualizamos 
metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


De sus nuevas funciones, elenco, equipo y palabras clave, debe extraer los tres actores más importantes, el director y las palabras clave asociadas con esa película.

Pero lo primero es lo primero, sus datos están presentes en forma de listas "en cadena". Debe convertirlos en una forma que le resulte útil.

In [111]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

A continuación, escribe funciones que le ayudarán a extraer la información necesaria de cada función.
Obtén el nombre del director de la función de equipo. Si el director no está en la lista, regreseNaN

In [112]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

A continuación, escribirá una función que devolverá los 3 elementos principales o la lista completa, lo que sea más. Aquí la lista se refiere a la cast, keywordsy genres.

In [113]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [114]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [115]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


El siguiente paso sería convertir los nombres y las instancias de palabras clave en minúsculas y eliminar todos los espacios entre ellos.

Eliminar los espacios entre palabras es un importante paso previo al procesamiento. Se hace para que su vectorizador no cuente al Johnny de "Johnny Depp" y "Johnny Galecki" como el mismo. Después de este paso de procesamiento, los actores mencionados anteriormente se representarán como "johnnydepp" y "johnnygalecki" y serán distintos a su vectorizador.

Otro buen ejemplo en el que el modelo podría generar la misma representación vectorial es "atasco de pan" y "atasco de tráfico". Por lo tanto, es mejor quitar cualquier espacio que esté presente.

La siguiente función lo hará exactamente por usted:

In [116]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [117]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

Ahora está en condiciones de crear su "sopa de metadatos", que es una cadena que contiene todos los metadatos que desea alimentar a su vectorizador (a saber, actores, director y palabras clave).

La create_soupfunción simplemente unirá todas las columnas requeridas por un espacio. Este es el último paso de preprocesamiento y la salida de esta función se alimentará al modelo de vector de palabras.

In [118]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [119]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [120]:
metadata[['soup']].head(2)

,soup
0,jealousy toy boy tomhanks timallen donrickles ...
1,boardgame disappearance basedonchildren'sbook ...


Los siguientes pasos son los mismos que hizo con su plot description based recommender. Una diferencia clave es que usa el en CountVectorizer()lugar de TF-IDF. Esto se debe a que no desea restar importancia a la presencia del actor / director si él o ella ha actuado o dirigido en relativamente más películas. No tiene mucho sentido intuitivo rebajarlos en este contexto.

La principal diferencia entre CountVectorizer()y TF-IDFes el componente de frecuencia inversa de documentos (IDF) que está presente en el último y no en el primero.

In [121]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

In [122]:
count_matrix.shape

(46628, 73881)

En la salida anterior, puede ver que hay 73,881 vocabularios en los metadatos que le proporcionó.

A continuación, utilizará cosine_similaritypara medir la distancia entre las incrustaciones.

In [123]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix[0:dim,0:dim], count_matrix[0:dim,0:dim])

In [124]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])

Ahora puede reutilizar su get_recommendations()función pasando la nueva cosine_sim2matriz como segundo argumento.

In [125]:
get_recommendations('Jumanji', cosine_sim2[0:dim,0:dim])

12                                 Balto
28             The City of Lost Children
32                      Wings of Courage
36                Across the Sea of Time
47                            Pocahontas
59            The Indian in the Cupboard
86                     Dunston Checks In
166    Free Willy 2 - The Adventure Home
254         A Kid in King Arthur's Court
263                          Major Payne
Name: title, dtype: object

In [126]:
get_recommendations('The Godfather', cosine_sim2[0:dim,0:dim])

1                         Jumanji
2                Grumpier Old Men
3               Waiting to Exhale
4     Father of the Bride Part II
5                            Heat
6                         Sabrina
7                    Tom and Huck
8                    Sudden Death
9                       GoldenEye
10         The American President
Name: title, dtype: object

¡Excelente! Verá que su recomendador ha logrado capturar más información debido a más metadatos y le ha dado mejores recomendaciones. Por supuesto, existen numerosas formas de experimentar con este sistema para mejorar las recomendaciones.

Algunas sugerencias:

Introducir un filtro de popularidad: este recomendador tomaría las 30 películas más similares, calcularía las calificaciones ponderadas (usando la fórmula IMDB de arriba), clasificaría las películas según esta calificación y devolvería las 10 películas principales.

Otros miembros del equipo: también se podrían incluir otros nombres de miembros del equipo, como guionistas y productores.

El peso creciente del director: para darle más peso al director, podría ser mencionado varias veces en la sopa para aumentar las puntuaciones de similitud de películas con el mismo director.